In [ ]:
import os 
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel, get_peft_model

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
lora_r = 8
lora_alpha = 8
lora_dropout = 0.2
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
fp16 = True
bf16 = False
num_train_epochs = 1
per_device_training_batch = 1
per_device_eval_batch = 1
gradient_accumilation = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.03
optim = "paged_adamw_8bit"
lr_scheduler = "cosine"
warmup_ratio = 0.01
group_by_length = True
sav_steps = 0
logging_steps = 25
max_sequene_length = 512

In [ ]:
tokenizer = AutoTokenizer.from_pretained(model_name,add_eos_token=True,add_bos_token=True,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")["train"]
eval_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")["eval"]

In [ ]:
compute_dtype = getattr(torch,bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(load_in_4bit=use_4bit,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_use_double_quant = use_nested_quant)
model = AutoModelForCausalLM.from_pretrained(model_name=model_name,
                                             quantization_config = bnb_config,
                                             device_map="auto") # will split the model across the GPUs


In [ ]:
peft_config = LoraConfig(lora_alpha=lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")
model = get_peft_model(model, peft_config)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits).reshape(-1,logits.shape[-1])
    labels = torch.tensor(labels).reshape(-1)
    mask = labels != 100

    masked_logits = logits[mask]
    masked_labels = labels[mask]

    predictions = torch.argmax(masked_logits, dim=-1)
    accuracy_metric = evaluate,load("accuracy")
    accuracy = accuracy_metric.compute(predictions=predictions, refrences = masked_labels)

    return accuracy

In [ ]:
training_arguments = SFTConfig(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    gradient_accumilation_steps = gradient_accumilation,
    optim = optim,
    save_steps = sav_steps,
    logging_steps = logging_steps,
    weight_decay = weight_decay,
    fp16 = fp16,
    bf16 = bf16,
    per_device_training_batch = per_device_training_batch,
    eval_strategy = "epoch",
    eval_accumulation_steps = 1,
    max_grad_norm = max_grad_norm,
    group_by_length = group_by_length,
    lr_scheduler_type = lr_scheduler
)

trainer = SFTrainer(
    model=model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    max_sequene_length = max_sequene_length
    compute_metrics = compute_metrics
)

result = trainer.train()